In [ ]:
j!pip install -U sklearn opencv-python numpy pillow pandas matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 60.9 MB 81 kB/s 
     |████████████████████████████████| 3.1 MB 38.7 MB/s 
     |████████████████████████████████| 11.2 MB 34.7 MB/s 
     |████████████████████████████████| 944 kB 49.5 MB/s 
  Attempting uninstall: pillow
    Found existing installation: Pillow 7.1.2
    Uninstalling Pillow-7.1.2:
      Successfully uninstalled Pillow-7.1.2
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following depende

In [ ]:
import os
import math
import string
import random
import shutil
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

In [ ]:
# Monta os arquivos do google drive
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
base_dir = Path('/content/drive/My Drive/TCC/')
#os.chdir('/content/drive/My Drive/TCC/')
base_dir.cwd()

Mounted at /content/drive/


PosixPath('/content')

In [ ]:
def get_string_bits(msg):
    msg_bytes = bytes(msg, "utf-8")
    result = []
    for byte in msg_bytes:
        byte_bits = []
        for i in range(8):
            bit = byte >> i & 1
            byte_bits.append(bit)
        byte_bits.reverse()
        result.extend(byte_bits)
    return result


def encode_msg(msg, start_x, start_y, image):
    msg_bits = get_string_bits(msg)
    pixels = image.load()
    x, y = start_x, start_y
    current_pixel_band = 0

    for bit in msg_bits:
        r, g, b = pixels[x, y]
        original_x, original_y = x, y
        if current_pixel_band == 0:
            r = (r & 0b11111110) | bit
            current_pixel_band = 1
        elif current_pixel_band == 1:
            g = (g & 0b11111110) | bit
            current_pixel_band = 2
        elif current_pixel_band == 2:
            b = (b & 0b11111110) | bit
            current_pixel_band = 0
            y += 1
            if y == image.size[1]:
                y = 0
                x += 1
        pixels[original_x, original_y] = (r, g, b)


def pixels_generator(image, start_x, start_y):
    pixels = image.load()
    x, y = start_x, start_y
    max_x, max_y = image.size
    while True:
        yield pixels[x, y]
        y += 1
        if y == max_y:
            y = 0
            x += 1
            if x == max_x:
                break


def pixels_bytes_generator(image, start_x, start_y):
    for pixel in pixels_generator(image, start_x, start_y):
        r, g, b = pixel
        yield r
        yield g
        yield b


def decode_msg(
    start_x, start_y, image, msg_length_chars
):
    result = ""
    bytes_generator = pixels_bytes_generator(image, start_x, start_y)
    for _ in range(msg_length_chars):
        char_byte = 0
        for i in range(8):
            pixel_byte = next(bytes_generator)
            bit = pixel_byte & 1
            char_byte |= bit << (7 - i)
        result += chr(char_byte)
    return result

In [ ]:
#TARGET_CAPACITIES = [10, 25, 50, 75]
TARGET_CAPACITIES = [100]
TARGET_START_X = 0
TARGET_START_Y = 0

total_image_lsbs = 255 * 255 * 3
total_capacity_bytes = math.floor(total_image_lsbs / 8)

dataset_folder = base_dir / 'base_png' / 'original_png'
#dataset_folder = base_dir / 'test_temp'
images = [file for file in dataset_folder.iterdir()]
print(f'Dataset has {len(images)} images')

for capacity in TARGET_CAPACITIES:
  random.shuffle(images)
  target_byte_count = math.floor(capacity / 100 * total_capacity_bytes)

  target_path = base_dir / f"lsb_{capacity}"
  if target_path.exists():
    shutil.rmtree(target_path)
  target_path.mkdir()

  images_len = len(images)
  images_len_half = images_len // 2
  for i, image_path in enumerate(images[:images_len_half]):
    print(f'Encoding image {i} - capacity {capacity}')
    img = Image.open(image_path)
    embed_str = ''.join(random.choice(string.ascii_letters) for x in range(target_byte_count))
    encode_msg(embed_str, TARGET_START_X, TARGET_START_Y, img)

    filename = image_path.stem

    output_path = target_path / f"encoded_{filename}.png"
    img.save(output_path)
  
  print(f"Copying remaining images - capacity {capacity}")
  for image_path in images[images_len_half:]:
    filename = image_path.stem
    output_path = target_path / f"{filename}.png"
    shutil.copy(image_path, output_path)


Dataset has 5150 images
Encoding image 0 - capacity 100
Encoding image 1 - capacity 100
Encoding image 2 - capacity 100
Encoding image 3 - capacity 100
Encoding image 4 - capacity 100
Encoding image 5 - capacity 100
Encoding image 6 - capacity 100
Encoding image 7 - capacity 100
Encoding image 8 - capacity 100
Encoding image 9 - capacity 100
Encoding image 10 - capacity 100
Encoding image 11 - capacity 100
Encoding image 12 - capacity 100
Encoding image 13 - capacity 100
Encoding image 14 - capacity 100
Encoding image 15 - capacity 100
Encoding image 16 - capacity 100
Encoding image 17 - capacity 100
Encoding image 18 - capacity 100
Encoding image 19 - capacity 100
Encoding image 20 - capacity 100
Encoding image 21 - capacity 100
Encoding image 22 - capacity 100
Encoding image 23 - capacity 100
Encoding image 24 - capacity 100
Encoding image 25 - capacity 100
Encoding image 26 - capacity 100
Encoding image 27 - capacity 100
Encoding image 28 - capacity 100
Encoding image 29 - capacity 